In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from functions import *
import numpy as np

In [ ]:
document1 = "This is a sample text containing some random words"
document2 = "This is a sample text with different random words"
s1 = shingle(document1, k=2)
s2 = shingle(document2, k=2)
compareSets(s1, s2)

In [ ]:
results = []
for _ in range(10000):
    minHash = MinHash()
    sig1 = minHash.create_signature(s1)
    sig2 = minHash.create_signature(s2)
    results.append(minHash.compareSets(sig1, sig2))
np.mean(results)